# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Pretty print in case you need it
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read in the CSV from WeatherPy to get the cities
cities_data_file = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(cities_data_file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,qinzhou,21.95,108.62,66.88,79,100,11.27,CN,1604001389
1,college,64.86,-147.80,17.60,78,90,5.82,US,1604001389
2,jamestown,42.10,-79.24,46.00,100,90,5.82,US,1604001389
3,provideniya,64.38,-173.30,19.40,62,0,5.99,RU,1604001290
4,sume,-7.67,-36.88,87.82,33,84,15.77,BR,1604001390


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Get the locations and Humidity, insure they are the correct datatype
locations = cities_df[["Lat","Lng"]].astype('float')
humidity = cities_df["Humidity"].astype('float')

# Configure gmaps using my key
gmaps.configure(api_key=g_key)

# Setup the figure, giving center lat, Lng, and zoom level
fig = gmaps.figure(center=(14.5994,-18.6731), zoom_level=2)

# Initialize the heat map layer
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=100,
    point_radius = 1.5   
)

# Add the layer
fig.add_layer(heat_layer)

# Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# I like a cool place, with less than 10 mph winds, humidity not too high, and less clouds 
ideal_vacation_weather = cities_df[
    (cities_df["Max Temp"] < 61) & 
    (cities_df["Max Temp"] > 44) &
    (cities_df["Wind Speed"] < 10) &
    (cities_df["Humidity"] < 50) &
    (cities_df["Cloudiness"] < 10)
]
ideal_vacation_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,coihaique,-45.58,-72.07,59.00,41,0,5.82,CL,1604001336
128,mount isa,-20.73,139.50,53.60,18,0,6.46,AU,1604001432
294,tabas,33.60,56.92,59.00,12,0,2.24,IR,1604001487
323,khalkhal,37.62,48.53,50.94,45,0,2.51,IR,1604001495
375,los lunas,34.81,-106.73,55.00,42,1,2.30,US,1604001517
392,farmington,36.73,-108.22,57.00,37,1,2.01,US,1604001524
470,takestan,36.07,49.70,53.60,40,0,4.70,IR,1604001547
513,brigham city,41.51,-112.02,55.40,26,1,5.39,US,1604001559
549,saveh,35.02,50.36,54.82,39,0,0.43,IR,1604001571
565,tatvan,38.51,42.28,57.29,36,1,9.22,TR,1604001576


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# I like to copy my data into the hotel df and then add the "Hotel Name" column
hotel_df = ideal_vacation_weather.copy(deep=True).reset_index(drop=True, inplace=False)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,coihaique,-45.58,-72.07,59.00,41,0,5.82,CL,1604001336,
1,mount isa,-20.73,139.50,53.60,18,0,6.46,AU,1604001432,
2,tabas,33.60,56.92,59.00,12,0,2.24,IR,1604001487,
3,khalkhal,37.62,48.53,50.94,45,0,2.51,IR,1604001495,
4,los lunas,34.81,-106.73,55.00,42,1,2.30,US,1604001517,
5,farmington,36.73,-108.22,57.00,37,1,2.01,US,1604001524,
6,takestan,36.07,49.70,53.60,40,0,4.70,IR,1604001547,
7,brigham city,41.51,-112.02,55.40,26,1,5.39,US,1604001559,
8,saveh,35.02,50.36,54.82,39,0,0.43,IR,1604001571,
9,tatvan,38.51,42.28,57.29,36,1,9.22,TR,1604001576,


In [6]:
# Set base url for google places API 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# We want hotels within 5000 meters
params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key,
}

# Loop through ideal locations and populate the hotels
for index, row in hotel_df.iterrows():
    
    # Grab lat and lng of the current row
    lat = row['Lat']
    lng = row["Lng"]
    
    # Set the logcation param for the call
    params['location'] = f"{lat},{lng}"
    
    # Call the places API
    response = requests.get(base_url, params=params)
    
    # Retrive the hotel list
    hotels = response.json()
    
    #pprint(hotels)
    
    # Make sure a hotel is present in the response
    # if not then set to "No Hotel Found"
    try:
        hotel_name = hotels['results'][0]['name']
    except (KeyError, IndexError):
        hotel_name = "No Hotel Found"
    
    # Set the hotel for this index location
    hotel_df.loc[index, "Hotel Name"] = hotel_name
    
hotel_df        

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,coihaique,-45.58,-72.07,59.00,41,0,5.82,CL,1604001336,Entre Cumbres Apart Hotel
1,mount isa,-20.73,139.50,53.60,18,0,6.46,AU,1604001432,Isa Hotel
2,tabas,33.60,56.92,59.00,12,0,2.24,IR,1604001487,Amir hotel
3,khalkhal,37.62,48.53,50.94,45,0,2.51,IR,1604001495,Hotel Almas
4,los lunas,34.81,-106.73,55.00,42,1,2.30,US,1604001517,Western Skies Inns & Suites
5,farmington,36.73,-108.22,57.00,37,1,2.01,US,1604001524,Casa Blanca Inn & Suites
6,takestan,36.07,49.70,53.60,40,0,4.70,IR,1604001547,رستوران و تالار هتل خوشبین
7,brigham city,41.51,-112.02,55.40,26,1,5.39,US,1604001559,Hampton Inn Brigham City
8,saveh,35.02,50.36,54.82,39,0,0.43,IR,1604001571,Sadra Hotel And Restaurant
9,tatvan,38.51,42.28,57.29,36,1,9.22,TR,1604001576,Tasar Royal Hotel


In [7]:
# Setup the figure, giving center lat, Lng, and zoom level#
fig = gmaps.figure(center=(15.5994,10.6731), zoom_level=2)

# Initialize the heat map layer like we did earlier
heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=100,
    point_radius = 1.25    
)

# Add the layer
fig.add_layer(heat_layer)

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Create the marker layer to place on top of the heat map
hotel_layer = gmaps.marker_layer(
    hotel_locations,
    info_box_content=hotel_info
)

# Add the markers
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))